In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tensorflow as tf

In [2]:
# 경로 설정
train_dir = 'C:\\WorkSpace\\Machine Learning\\dataset_v2\\dataset'
test_dir = 'C:\\WorkSpace\\Machine Learning\\testset(300x300)\\testset(300x300)'

# ImageDataGenerator 설정 (데이터 증강 없이 단순 스케일링만 적용)
test_datagen = ImageDataGenerator(rescale=1.0/255)
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
# 배치 단위로 이미지를 불러오기
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),   # 이미지를 300x300으로 리사이즈
    batch_size=64,
    class_mode='sparse',      # 레이블을 정수형으로 반환
    shuffle=True              # 데이터 셔플 (섞기)
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='sparse',
    shuffle=False             # 테스트 데이터는 셔플하지 않음
)

Found 77373 images belonging to 100 classes.
Found 10587 images belonging to 100 classes.


In [5]:
# 간단한 모델 생성
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dense(100, activation='softmax')  # 100개의 클래스로 분류
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow import keras

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convent_from_scratch.h5",
        save_best_only=True,
        monitor="val_loss")
]

history = model.fit(
    train_generator,
    epochs=30,
    validation_data=test_generator,
    callbacks=callbacks
)

Epoch 1/30
1209/1209 [==============================] - 1309s 1s/step - loss: 3.1692 - accuracy: 0.2608 - val_loss: 2.8521 - val_accuracy: 0.2772
Epoch 2/30
1209/1209 [==============================] - 1547s 1s/step - loss: 2.4051 - accuracy: 0.3952 - val_loss: 2.6260 - val_accuracy: 0.3167
Epoch 3/30
1209/1209 [==============================] - 1551s 1s/step - loss: 2.0857 - accuracy: 0.4597 - val_loss: 2.1536 - val_accuracy: 0.4217
Epoch 4/30
1209/1209 [==============================] - 1547s 1s/step - loss: 1.9056 - accuracy: 0.4996 - val_loss: 1.9415 - val_accuracy: 0.4722
Epoch 5/30
1209/1209 [==============================] - 1550s 1s/step - loss: 1.7878 - accuracy: 0.5248 - val_loss: 1.8367 - val_accuracy: 0.4973
Epoch 6/30
1209/1209 [==============================] - 1549s 1s/step - loss: 1.6931 - accuracy: 0.5460 - val_loss: 1.7535 - val_accuracy: 0.5191
Epoch 7/30
1209/1209 [==============================] - 1549s 1s/step - loss: 1.6205 - accuracy: 0.5646 - val_loss: 1.6329 -

In [ ]:
import matplotlib.pyplot as plt

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy)+1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.legend()
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")